In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder,StandardScaler
import tensorflow as tf
from sklearn.model_selection import train_test_split
import numpy as np


In [2]:
data = pd.read_csv("dataset_final.csv")
data

,Sold_price,Squarefootage,Type,Style,Community,Municipality District,longitude,latitude,Bedrooms,Dens,Bathrooms,Kitchens,Rooms,Parking Total
0,205000,950.0,Condo Apartment,Apartment,Black Creek,W05,-79.519841,43.769643,2,0,1,1,5,1.0
1,230000,550.0,Co-op / Co-Ownership Apartment,Apartment,West Hill,E10,-79.182497,43.775476,1,0,1,1,4,0.0
2,299000,1100.0,Condo Townhouse,Stacked Townhouse,Glenfield-Jane Heights,W05,-79.515943,43.749223,3,0,2,1,6,1.0
3,275000,250.0,Condo Apartment,Apartment,Downsview-Roding-CFB,W05,-79.481416,43.725043,0,0,1,1,3,1.0
4,305888,1100.0,Condo Apartment,Apartment,Glenfield-Jane Heights,W05,-79.515943,43.749223,3,0,2,1,6,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6212,6198000,4250.5,Detached,2-Storey,Forest Hill South,C03,-79.418061,43.696341,3,2,5,1,11,3.0
6213,7000000,5000.0,Detached,2-Storey,Forest Hill South,C03,-79.417288,43.691219,4,2,8,1,9,6.0
6214,7150000,5000.0,Detached,2-Storey,Bridle Path-Sunnybrook-York Mills,C12,-79.390151,43.743789,4,1,8,1,10,7.0
6215,6800000,5000.0,Detached,2-Storey,Forest Hill South,C03,-79.415512,43.700019,4,1,7,1,9,6.0


In [3]:
# defining a callback function for early stopping of model
def get_callbacks():
  return [
    tf.keras.callbacks.EarlyStopping(monitor='val_mean_absolute_error', patience=50, verbose=1),
  ]

In [4]:
# Creating custom R^2 performance metric for keras models
def r2(y_true,y_pred):
    from keras import backend as K
    SS_res = K.sum(K.square(y_true-y_pred))
    SS_tot = K.sum(K.square(y_true - K.mean(y_true)))
    return (1 - SS_res/(SS_tot + K.epsilon()))

In [5]:



X = data.drop(['Sold_price','longitude','latitude'],axis=1)
Y = data.drop(['Squarefootage','Type','Style','Community','Municipality District','Bedrooms','Dens','Bathrooms','Kitchens','Rooms','Parking Total','longitude','latitude'],axis=1)




In [6]:
Y

,Sold_price
0,205000
1,230000
2,299000
3,275000
4,305888
...,...
6212,6198000
6213,7000000
6214,7150000
6215,6800000


In [7]:
X

,Squarefootage,Type,Style,Community,Municipality District,Bedrooms,Dens,Bathrooms,Kitchens,Rooms,Parking Total
0,950.0,Condo Apartment,Apartment,Black Creek,W05,2,0,1,1,5,1.0
1,550.0,Co-op / Co-Ownership Apartment,Apartment,West Hill,E10,1,0,1,1,4,0.0
2,1100.0,Condo Townhouse,Stacked Townhouse,Glenfield-Jane Heights,W05,3,0,2,1,6,1.0
3,250.0,Condo Apartment,Apartment,Downsview-Roding-CFB,W05,0,0,1,1,3,1.0
4,1100.0,Condo Apartment,Apartment,Glenfield-Jane Heights,W05,3,0,2,1,6,1.0
...,...,...,...,...,...,...,...,...,...,...,...
6212,4250.5,Detached,2-Storey,Forest Hill South,C03,3,2,5,1,11,3.0
6213,5000.0,Detached,2-Storey,Forest Hill South,C03,4,2,8,1,9,6.0
6214,5000.0,Detached,2-Storey,Bridle Path-Sunnybrook-York Mills,C12,4,1,8,1,10,7.0
6215,5000.0,Detached,2-Storey,Forest Hill South,C03,4,1,7,1,9,6.0


In [8]:
for i in range(len(X.columns)):
    if type(X.iloc[0,i])==str:
        X[X.columns[i]] = LabelEncoder().fit_transform(X[X.columns[i]])
    else:
        p = X[X.columns[i]]
        if p.min()==p.max():
            continue
        p -= p.min()
        p /= p.max()
        X[X.columns[i]] = p

In [9]:
yScaler = StandardScaler()
yScaler.fit(Y)
Y = yScaler.transform(Y)





In [10]:
Xtr,Xts,Ytr,Yts = train_test_split(X,Y,test_size=0.1, random_state=42)


In [11]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=[X.shape[1]]),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10),
  tf.keras.layers.Dense(1)
])

2021-08-24 18:14:35.305018: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-08-24 18:14:35.305316: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.mean_squared_error,
              metrics=['mean_squared_error'])

In [13]:
model.fit(X, Y, epochs=500)

2021-08-24 18:14:36.217546: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


Epoch 1/500
195/195 [==============================] - 0s 532us/step - loss: 19.6759 - mean_squared_error: 19.6759
Epoch 2/500
195/195 [==============================] - 0s 486us/step - loss: 1.4481 - mean_squared_error: 1.4481
Epoch 3/500
195/195 [==============================] - 0s 471us/step - loss: 0.8991 - mean_squared_error: 0.8991
Epoch 4/500
195/195 [==============================] - 0s 458us/step - loss: 0.6263 - mean_squared_error: 0.6263
Epoch 5/500
195/195 [==============================] - 0s 471us/step - loss: 0.4330 - mean_squared_error: 0.4330
Epoch 6/500
195/195 [==============================] - 0s 465us/step - loss: 0.4526 - mean_squared_error: 0.4526
Epoch 7/500
195/195 [==============================] - 0s 532us/step - loss: 0.4508 - mean_squared_error: 0.4508
Epoch 8/500
195/195 [==============================] - 0s 503us/step - loss: 0.4371 - mean_squared_error: 0.4371
Epoch 9/500
195/195 [==============================] - 0s 490us/step - loss: 0.4107 - mean_squ

Epoch 73/500
195/195 [==============================] - 0s 482us/step - loss: 0.2065 - mean_squared_error: 0.2065
Epoch 74/500
195/195 [==============================] - 0s 469us/step - loss: 0.2509 - mean_squared_error: 0.2509
Epoch 75/500
195/195 [==============================] - 0s 458us/step - loss: 0.2350 - mean_squared_error: 0.2350
Epoch 76/500
195/195 [==============================] - 0s 476us/step - loss: 0.2198 - mean_squared_error: 0.2198
Epoch 77/500
195/195 [==============================] - 0s 511us/step - loss: 0.2515 - mean_squared_error: 0.2515
Epoch 78/500
195/195 [==============================] - 0s 461us/step - loss: 0.2086 - mean_squared_error: 0.2086
Epoch 79/500
195/195 [==============================] - 0s 483us/step - loss: 0.2334 - mean_squared_error: 0.2334
Epoch 80/500
195/195 [==============================] - 0s 477us/step - loss: 0.2249 - mean_squared_error: 0.2249
Epoch 81/500
195/195 [==============================] - 0s 458us/step - loss: 0.2483 - m

195/195 [==============================] - 0s 550us/step - loss: 0.2225 - mean_squared_error: 0.2225
Epoch 145/500
195/195 [==============================] - 0s 524us/step - loss: 0.2140 - mean_squared_error: 0.2140
Epoch 146/500
195/195 [==============================] - 0s 531us/step - loss: 0.2313 - mean_squared_error: 0.2313
Epoch 147/500
195/195 [==============================] - 0s 551us/step - loss: 0.2254 - mean_squared_error: 0.2254
Epoch 148/500
195/195 [==============================] - 0s 545us/step - loss: 0.2259 - mean_squared_error: 0.2259
Epoch 149/500
195/195 [==============================] - 0s 588us/step - loss: 0.2284 - mean_squared_error: 0.2284
Epoch 150/500
195/195 [==============================] - 0s 562us/step - loss: 0.2036 - mean_squared_error: 0.2036
Epoch 151/500
195/195 [==============================] - 0s 481us/step - loss: 0.2130 - mean_squared_error: 0.2130
Epoch 152/500
195/195 [==============================] - 0s 487us/step - loss: 0.2061 - mean_s

195/195 [==============================] - 0s 497us/step - loss: 0.2155 - mean_squared_error: 0.2155
Epoch 216/500
195/195 [==============================] - 0s 474us/step - loss: 0.2128 - mean_squared_error: 0.2128
Epoch 217/500
195/195 [==============================] - 0s 475us/step - loss: 0.2027 - mean_squared_error: 0.2027
Epoch 218/500
195/195 [==============================] - 0s 483us/step - loss: 0.2338 - mean_squared_error: 0.2338
Epoch 219/500
195/195 [==============================] - 0s 478us/step - loss: 0.2323 - mean_squared_error: 0.2323
Epoch 220/500
195/195 [==============================] - 0s 468us/step - loss: 0.2811 - mean_squared_error: 0.2811
Epoch 221/500
195/195 [==============================] - 0s 541us/step - loss: 0.1884 - mean_squared_error: 0.1884
Epoch 222/500
195/195 [==============================] - 0s 463us/step - loss: 0.1914 - mean_squared_error: 0.1914
Epoch 223/500
195/195 [==============================] - 0s 469us/step - loss: 0.2191 - mean_s

195/195 [==============================] - 0s 474us/step - loss: 0.1968 - mean_squared_error: 0.1968
Epoch 287/500
195/195 [==============================] - 0s 469us/step - loss: 0.2297 - mean_squared_error: 0.2297
Epoch 288/500
195/195 [==============================] - 0s 480us/step - loss: 0.2056 - mean_squared_error: 0.2056
Epoch 289/500
195/195 [==============================] - 0s 474us/step - loss: 0.1982 - mean_squared_error: 0.1982
Epoch 290/500
195/195 [==============================] - 0s 469us/step - loss: 0.1924 - mean_squared_error: 0.1924
Epoch 291/500
195/195 [==============================] - 0s 485us/step - loss: 0.2246 - mean_squared_error: 0.2246
Epoch 292/500
195/195 [==============================] - 0s 514us/step - loss: 0.2590 - mean_squared_error: 0.2590
Epoch 293/500
195/195 [==============================] - 0s 483us/step - loss: 0.2153 - mean_squared_error: 0.2153
Epoch 294/500
195/195 [==============================] - 0s 465us/step - loss: 0.1984 - mean_s

195/195 [==============================] - 0s 482us/step - loss: 0.2116 - mean_squared_error: 0.2116
Epoch 358/500
195/195 [==============================] - 0s 477us/step - loss: 0.2079 - mean_squared_error: 0.2079
Epoch 359/500
195/195 [==============================] - 0s 464us/step - loss: 0.1804 - mean_squared_error: 0.1804
Epoch 360/500
195/195 [==============================] - 0s 469us/step - loss: 0.2190 - mean_squared_error: 0.2190
Epoch 361/500
195/195 [==============================] - 0s 526us/step - loss: 0.1934 - mean_squared_error: 0.1934
Epoch 362/500
195/195 [==============================] - 0s 486us/step - loss: 0.2094 - mean_squared_error: 0.2094
Epoch 363/500
195/195 [==============================] - 0s 456us/step - loss: 0.1954 - mean_squared_error: 0.1954
Epoch 364/500
195/195 [==============================] - 0s 464us/step - loss: 0.2113 - mean_squared_error: 0.2113
Epoch 365/500
195/195 [==============================] - 0s 485us/step - loss: 0.2222 - mean_s

195/195 [==============================] - 0s 530us/step - loss: 0.2591 - mean_squared_error: 0.2591
Epoch 429/500
195/195 [==============================] - 0s 523us/step - loss: 0.2036 - mean_squared_error: 0.2036
Epoch 430/500
195/195 [==============================] - 0s 506us/step - loss: 0.1961 - mean_squared_error: 0.1961
Epoch 431/500
195/195 [==============================] - 0s 506us/step - loss: 0.2030 - mean_squared_error: 0.2030
Epoch 432/500
195/195 [==============================] - 0s 480us/step - loss: 0.2250 - mean_squared_error: 0.2250
Epoch 433/500
195/195 [==============================] - 0s 500us/step - loss: 0.2036 - mean_squared_error: 0.2036
Epoch 434/500
195/195 [==============================] - 0s 544us/step - loss: 0.1984 - mean_squared_error: 0.1984
Epoch 435/500
195/195 [==============================] - 0s 489us/step - loss: 0.1916 - mean_squared_error: 0.1916
Epoch 436/500
195/195 [==============================] - 0s 494us/step - loss: 0.2006 - mean_s

195/195 [==============================] - 0s 510us/step - loss: 0.1887 - mean_squared_error: 0.1887
Epoch 500/500
195/195 [==============================] - 0s 466us/step - loss: 0.2033 - mean_squared_error: 0.2033


In [14]:
def prediction(rows):
    result = model.predict(rows)
    scaledRes = []
    for i in result:
        scaledRes += [yScaler.inverse_transform(i)]
    return scaledRes

In [15]:
model.evaluate(Xts, Yts, batch_size=128, verbose =1)

5/5 [==============================] - 0s 792us/step - loss: 0.1798 - mean_squared_error: 0.1798


[0.17980600893497467, 0.17980600893497467]

In [18]:
x = np.array(X.iloc[0])

In [19]:
x

array([ 0.14736842,  1.        ,  4.        , 13.        , 29.        ,
        0.22222222,  0.        ,  0.08333333,  0.16666667,  0.16666667,
        0.06666667])

In [21]:
X.iloc[0]

Squarefootage             0.147368
Type                      1.000000
Style                     4.000000
Community                13.000000
Municipality District    29.000000
Bedrooms                  0.222222
Dens                      0.000000
Bathrooms                 0.083333
Kitchens                  0.166667
Rooms                     0.166667
Parking Total             0.066667
Name: 0, dtype: float64

In [25]:


x = np.array(X.iloc[0])

x = x.reshape(1,11)
x

array([[ 0.14736842,  1.        ,  4.        , 13.        , 29.        ,
         0.22222222,  0.        ,  0.08333333,  0.16666667,  0.16666667,
         0.06666667]])

In [26]:
result = model.predict(x)
print(yScaler.inverse_transform(result))

[[518416.12]]
